In [9]:
import pandas as pd

In [10]:
df1 = pd.read_csv("C:/Users/YAP/Downloads/preprocessing_data.csv")

In [11]:
# 정상 : 불량 = 9 : 1 로 Imbalanced 한 데이터이다 

df1['target_binom'].value_counts()

0.0    688
1.0     64
Name: target_binom, dtype: int64

In [12]:
df1.columns

Index(['Unnamed: 0', 'Ox_Chamber', 'process', 'type', 'Temp_OXid', 'Vapor',
       'ppm', 'Pressure', 'Oxid_time', 'thickness', 'No_Die', 'Reinforcement',
       'Unnamed: 0_x', 'photo_soft_Chamber', 'process 2', 'resist_target',
       'N2_HMDS', 'pressure_HMDS', 'temp_HMDS', 'temp_HMDS_bake',
       'time_HMDS_bake', 'spin1', 'spin2', 'spin3', 'photoresist_bake',
       'temp_softbake', 'time_softbake', 'lithography_Chamber', 'Line_CD',
       'UV_type', 'Wavelength', 'Resolution', 'Energy_Exposure', 'Range_check',
       'Unnamed: 0_y', 'Etching_Chamber', 'Process 3', 'Temp_Etching',
       'Source_Power', 'Selectivity', 'Thin Film 4', 'Thin Film 3',
       'Thin Film 2', 'Thin Film 1', 'Etching_rate', 'Chamber_Num', 'process4',
       'Flux60s', 'Flux90s', 'Flux160s', 'Flux480s', 'Flux840s',
       'input_Energy', 'Temp_implantation', 'Furance_Temp', 'RTA_Temp',
       'Target', 'Error_message', 'target_binom', 'Chamber_Route'],
      dtype='object')

In [13]:
Y = df1['target_binom']
X = df1[[ 'Vapor', 'resist_target', 'Line_CD', 'UV_type','Etching_rate','thickness','Temp_Etching']]

In [3]:
!pip install --user imblearn  

In [1]:
!pip uninstall scikit-learn


^C


In [2]:
!pip install scikit-learn==1.2.2

     ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
     - -------------------------------------- 0.3/8.3 MB 5.2 MB/s eta 0:00:02
     -- ------------------------------------- 0.6/8.3 MB 5.8 MB/s eta 0:00:02
     ---- ----------------------------------- 0.8/8.3 MB 5.9 MB/s eta 0:00:02
     ----- ---------------------------------- 1.1/8.3 MB 5.9 MB/s eta 0:00:02
     ------- -------------------------------- 1.5/8.3 MB 6.1 MB/s eta 0:00:02
     -------- ------------------------------- 1.8/8.3 MB 6.2 MB/s eta 0:00:02
     --------- ------------------------------ 2.0/8.3 MB 6.1 MB/s eta 0:00:02
     ----------- ---------------------------- 2.3/8.3 MB 6.2 MB/s eta 0:00:01
     ------------ --------------------------- 2.6/8.3 MB 6.1 MB/s eta 0:00:01
     ------------- -------------------------- 2.8/8.3 MB 6.0 MB/s eta 0:00:01
     -------------- ------------------------- 3.1/8.3 MB 6.0 MB/s eta 0:00:01
     ---------------- ----------------------- 3.4/8.3 MB 6.0 MB/s eta 0

In [6]:
# 학습 데이터와 검증데이터 분할
from sklearn.model_selection import train_test_split
# 특성공학 + 학습 Pipe 구성
# 연속형 데이터와 범주형 데이터를 따로 처리하는 Pipe
from sklearn.compose import make_column_transformer
# 각각의 처리기법을 파이프 형태로 구성
from imblearn.pipeline import make_pipeline

# 연속형 데이터 전처리
# 결측값을 주변에 있는 데이터와 유사한 데이터로 채운다 -> KNN Imputer 
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

# 범주형 데이터 전처리 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Imbalanced Data Sampling -> 데이터 개수가 적으니 Fake data를 생성해 비율을 맞춘다. (Over_sampling)
from imblearn.over_sampling import SMOTE

#학습 알고리즘으로 학습
from sklearn.tree import DecisionTreeClassifier
# 교차검증 + Hyperparameter Tuning 
from sklearn.model_selection import GridSearchCV
# 평가지표
from sklearn.metrics import classification_report




In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=1234)

In [18]:
numeric_list = ['resist_target','Line_CD','Etching_rate','thickness','Temp_Etching']
category_list = ['Vapor', 'UV_type']

In [20]:
numeric_pipe = make_pipeline(KNNImputer(), MinMaxScaler())  # 연속형데이터 전처리
category_pipe = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())  #범주형데이터 전처리

#연속형, 범주형 데이터 전처리 pipeline을 합침 
preprocessing_pipe = make_column_transformer((numeric_pipe, numeric_list),(category_pipe, category_list))   

In [23]:
pipe_model = make_pipeline(preprocessing_pipe, SMOTE(), DecisionTreeClassifier())

In [25]:
pipe_model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('knnimputer',
                                                                   KNNImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['resist_target', 'Line_CD',
                                                   'Etching_rate', 'thickness',
                                                   'Temp_Etching']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['Vapor', 'UV_type'])])),
                ('smote', SMOTE()),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [31]:
hyperparameter = {'decisiontreeclassifier__max_depth':range(5,11), 
                 'decisiontreeclassifier__min_samples_split':range(5,11),
                 'decisiontreeclassifier__min_samples_leaf':range(5,11)}

grid_model = GridSearchCV(pipe_model, param_grid=hyperparameter, cv=3, n_jobs= -1, scoring='f1')
# n_jobs 는 cpu가 처리를 빨리할 수 있도록 하는 Option 
# scoring = 'f1' 은 f1 score을 기준으로 best model을 찾기 위해 설정하는 옵션 

grid_model.fit(X_train, Y_train)


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('knnimputer',
                                                                                          KNNImputer()),
                                                                                         ('minmaxscaler',
                                                                                          MinMaxScaler())]),
                                                                         ['resist_target',
                                                                          'Line_CD',
                                                                          'Etching_rate',
                                                                          'thickness',
                                                                          'Temp_Etching']),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder())]),
                                                                         ['Vapor',
                                                                          'UV_type'])])),
                                       ('smote', SMOTE()),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid={'decisiontreeclassifier__max_depth': range(5, 11),
                         'decisiontreeclassifier__min_samples_leaf': range(5, 11),
                         'decisiontreeclassifier__min_samples_split': range(5, 11)},
             scoring='f1')

In [33]:
best_model = grid_model.best_estimator_

In [49]:
def evaluation_cla(model):
    Y_train_pred = model.predict(X_train)
    Y_test_pred = model.predict(X_test)
    print('학습 능력 평가 : ')
    print(classification_report(Y_train, Y_train_pred))
    print('일반화 능력 평가 : ')
    print(classification_report(Y_test, Y_test_pred))
    roc_auc= roc_auc_score(Y_test, Y_test_pred)
    print('ROC AUC Score :', roc_auc)

In [50]:
evaluation_cla(best_model)

학습 능력 평가 : 
              precision    recall  f1-score   support

         0.0       0.98      0.93      0.96       516
         1.0       0.54      0.83      0.66        48

    accuracy                           0.93       564
   macro avg       0.76      0.88      0.81       564
weighted avg       0.95      0.93      0.93       564

일반화 능력 평가 : 
              precision    recall  f1-score   support

         0.0       0.94      0.87      0.90       172
         1.0       0.21      0.38      0.27        16

    accuracy                           0.82       188
   macro avg       0.57      0.62      0.58       188
weighted avg       0.87      0.82      0.85       188

ROC AUC Score : 0.6206395348837209


In [43]:
from sklearn.metrics import roc_auc_score

**모델을 파일 형태로 저장**

In [39]:
import pickle

In [40]:
pickle.dump(best_model, open('model.sav','wb'))